(ch_gpu_arch)=
# GPU 架构

高端 GPU 通常比高端 CPU 提供更好的性能。尽管 GPU 和 CPU 之间的术语和编程范式有所不同，但它们的架构是相似的，GPU 拥有更宽的 SIMD 宽度和更多的核。在本节中，将简要回顾 GPU 架构与 {ref}`ch_cpu_arch` 中介绍的 CPU 架构的比较。

(FIXME, changed from V100 to T4 in CI..., also changed cpu...)

使用的系统有 [Tesla T4](https://www.nvidia.com/content/dam/en-zz/Solutions/design-visualization/technologies/turing-architecture/NVIDIA-Turing-Architecture-Whitepaper.pdf) GPU，是基于图灵架构的。Tesla T4 是基于图灵架构的 GPU 卡，用于深度学习模型推理加速。

本教程使用：

In [1]:
!nvidia-smi -q -i 0 | grep "Product Name"

    Product Name                          : NVIDIA GeForce RTX 2080 Ti


## 流式多核处理器

流式多核处理器（streaming multiprocessor，简称 SM）大致相当于 CPU 核心。T4 使用的 SM 如下所示:numref: ' fig_gpu_sm '。

```{figure} ../img/gpu_sm.svg
A streaming multiprocessor in Tesla T4
```

可以看到，SM 被划分为 4 个处理块。在每个块中，有 16 个算术单元（AU）用于处理 float32 数字，也称为 FP32 CUDA 核。总的来说，SM 有 64 个 FP32 AUs，它们每次能够执行 64 个 float32 算子（例如 FMA）。除了寄存器文件和指令加载/解码器，SM 有 8 个张量 core。每个张量核都能够在每次执行 $4\times 4$  float16（或 int8/int4）矩阵乘法。所以每个，可以称之为 FP16 AU，每个时钟计算 $2\times 4^3=128$ 个算子。值得注意的是，在本章中不会使用张量核。将在后面的章节讨论如何使用它。

另外，SM 只有与 CPU 相似的 L1 cache。但是，可以将此存储作为在 SM 上运行的所有线程的共享内存。我们知道缓存由硬件和操作系统控制，而我们可以显式地分配和回收共享内存上的空间，这为我们进行性能优化提供了更大的灵活性。

## GPU 架构简介

特斯拉 T4 卡包含 40 条 SMs，所有 SM 共享 6MB 的 L2 缓存。它还附带了连接到处理器的 16GB 高带宽内存（GDDR6）。

```{figure} ../img/gpu_t4.svg
The Tesla T4 Architecture
```

(tab_cpu_gpu_compare)=
|Hardware | Intel E5-2686 v4 | Tesla P100 | Tesla V100 | Tesla T4 |
|------|------|------|------|------|
| Clock rate (GHz) | **3** | 1.48 | 1.53 | 1.59 |
| # cores | 16 | 56 | **80** | 40 |
| # FP64 AUs per core | 4 | **32** | **32** | x |
| # FP32 AUs per core | 8 | **64** | **64** | **64** |
| # FP16 AUs per core | x | x$^*$ | **8** | **8** |
| cache per core (KB) | **320** | 64 | 128 | 64 |
| shared cache (MB)| **45** | 4 | 6 | 6 |
| Memory (GB) | **240** | 16 | 16 | 16 |
| Max memory bandwidth (GB/sec) | 72 | 732 | **900** | 300 |
| FP64 TFLOPS | 0.38 | 4.7 | **7.8** | x |
| FP32 TFLOPS | 0.77 | 9.3 | **15.7** | 8.1 |
| FP16 TFLOPS | x | 18.7 | **125.3** | 65 |

## 小结

- GPU 在概念上与 CPU 的架构类似，但速度要快得多。
